# W261 Final Project

#### *Anusha Munjuluri, Arvindh Ganesan, Kim Vignola, Christina Papadimitriou*

### Notebook Set-up

In [582]:
# imports
import re
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [583]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [584]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "final_project"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

## 1. Question Formulation


### Background 

The following analysis is based on a Kaggle dataset from Criteo, an internet advertising company focused on retargeting. Criteo's goal is to increase online clickthrough rates among consumers who have previously visited an advertiser's website. This information will be used by Criteo to more efficiently provide the right ads to the right people. Optimizing the retargeting process not only helps advertisers become more efficient in terms of how they spend their dollars, but also it reduces clutter for consumers who do not want to be "followed" by ads for irrelevant products (or ones they may have already purchased!).

There are 13 numerical features and 26 categorical features in this dataset. Our goal is to create a model that will most accurately predict clickthroughs (label = 1). It is likely that these features represent characterstics about consumer behavior (history of clickthroughs, site visitiation, etc.), the ads themselves (product, creative approach, placement, etc.) and general metrics such as the date the ad was published. Since there is no visibility into what each feature represents, however, our challenge is to make our predictions based on the data alone. With over 6 million records, this will require a scalable approach.

### Key Questions

* Which machine learning approach not only provides the highest accuracy in predicting clickthroughs, but is also scalable enough to be useful in a production environment? As internet patterns and product choices change rapidly, the ideal model should be updated daily to update the following day's retargeting model.  - Note: not sure I fully answered this piece:  *** Preview what level of performanceyour model would need to achieve to be practically useful ***

* Which features are most important in predicting clickthroughs? Having this information can help Criteo focus on the metrics that are most critical to their product.

* With 39 features, there is a high risk of overfitting. We should identify a model that provides an optimal tradeoff between bias and variance.


## 2. Algorithm Explanation

### Data Loading and Pre-Processing

In [ ]:
# take a look at the data
!head -n 1 data/train.txt

In [651]:
# load the data
fullTrainRDD = sc.textFile('data/train.txt')
testRDD = sc.textFile('data/test.txt')

FIELDS = ['I1','I2','I3','I4','I5','I6','I7','I8','I9','I10','I11','I12','I13',
          'C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14',
          'C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26','Label']

In [624]:
# number of rows in train/test data
print(f"Number of records in train data: {fullTrainRDD.count()} ...")
print(f"Number of records in test data: {testRDD.count()} ...")

Number of records in train data: 45840617 ...
Number of records in test data: 6042135 ...


In [586]:
# Generate 80/20 (pseudo)random train/test split 
trainRDD, heldOutRDD = fullTrainRDD.randomSplit([0.8,0.2], seed = 1)
print(f"... held out {heldOutRDD.count()} records for evaluation and assigned {trainRDD.count()} for training.")

... held out 9167871 records for evaluation and assigned 36672746 for training.


In [652]:
toyRDD_train, trainRDD2 = trainRDD.randomSplit([0.001,0.999], seed = 2)
toyRDD_test, mainRDD_test = heldOutRDD.randomSplit([0.001,0.999], seed = 2)

In [653]:
#toyRDD_train.take(1)
toyRDD_train.take(1)

['1\t5\t2\t\t\t1382\t17\t78\t25\t76\t0\t9\t\t\t05db9164\t942f9a8d\t56472604\t53a5d493\t25c83c98\t\t49b74ebc\t6c41e35e\ta73ee510\te113fc4b\tc4adf918\t08531bcb\t85dbe138\t1adce6ef\tae97ecc3\t76b06ec3\te5ba7672\t1f868fdd\t9437f62f\ta458ea53\tff4c70b8\t\t32c7478e\tda89b7d5\t7a402766\tc7beb94e']

In [654]:
# helper functions
def parse(line):
    """
    Map line --> tuple of (features, label)
    """
    fields = np.array(line.split('\t'))
    features,label = fields[1:14], fields[0]
    return(features, label)

def edit_data_types(line):
    """
    Map tuple of (features, label) --> tuple of (formated features, label)
    
    * '' is replaced with 'null'
    * numerical fields are converted to integers
    """
    features, label = line[0], line[1]
    formated_features = []
    for i, value in enumerate(features):
        if value == '':
            formated_features.append(np.nan)
        else:
            if i < 13:
                formated_features.append(float(value)) 
            else:
                formated_features.append(float(value))
    return (formated_features, label)

In [655]:
#trainRDDCached = trainRDD.map(parse).map(edit_data_types).cache()
toyRDD_train1 = toyRDD_train.map(parse).map(edit_data_types)
toyRDD_test2 = toyRDD_test.map(parse).map(edit_data_types)

In [656]:
print(toyRDD_train1.count())
#print(toyRDD_test1.count())

36451


In [657]:
sample = np.array(toyRDD_train1.map(lambda x: np.append(x[0], [x[1]])).takeSample(False, 1000))
sample_df = pd.DataFrame(np.array(sample), columns = ['I1','I2','I3','I4','I5','I6','I7','I8','I9','I10','I11','I12','I13', 'Label'])

In [658]:
columns = (['I1','I2','I3','I4','I5','I6','I7','I8','I9','I10','I11','I12','I13', 'Label'])
#columns = ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13']
sample_numeric = sample_df.reindex(columns=columns)
sample_numeric[columns] = sample_numeric[columns].astype(np.float)

In [659]:
"""Get means and standard deviations. Ideally we should do this in the RDD vs. pandas"""

means = []
stdevs = []

for i in sample_numeric.columns[0:13]:
    mean = np.nanmean(sample_numeric[i])
    means.append(mean)
    std = np.nanstd(sample_numeric[i])
    stdevs.append(std)
        
print(means)
print(stdevs)


[3.4376199616122842, 90.14, 19.89001264222503, 6.916349809885932, 17246.35677352637, 124.63518758085381, 14.970772442588727, 12.226226226226226, 106.30480167014613, 0.6084452975047985, 2.5260960334029225, 0.6375, 8.043092522179975]
[10.984391530315705, 323.5902044252885, 79.83880276011938, 8.267584408314738, 55493.864000837486, 288.36392470232033, 64.45374433092296, 12.987052645203304, 205.91002503807812, 0.6956443052357069, 5.146988458022784, 2.40265694943466, 10.356297402365538]


In [660]:
"""Get medians for each class. Ideally we should do this in the RDD vs. pandas"""

median1 = np.array(sample_numeric[sample_numeric['Label'] == 1.0].median().tolist())
print(median1)

median0 = np.array(sample_numeric[sample_numeric['Label'] == 0.0].median().tolist())
print(median0)

[2.000e+00 2.000e+00 5.000e+00 4.000e+00 1.516e+03 1.900e+01 5.000e+00
 6.000e+00 4.000e+01 1.000e+00 2.000e+00 0.000e+00 3.000e+00 1.000e+00]
[1.000e+00 3.000e+00 6.000e+00 4.000e+00 3.506e+03 4.300e+01 3.000e+00
 7.000e+00 3.900e+01 0.000e+00 1.000e+00 0.000e+00 4.000e+00 0.000e+00]


In [661]:
# helper functions
def parse(line):
    """
    Map line --> tuple of (features, label)
    """
    fields = np.array(line.split('\t'))
    features,label = fields[1:14], fields[0]
    return(features, label)

def update_nans(line):
    """
    Replace missing values with meidans for each label    
    """
    
    #median1 = np.array([2.0, 3.5, 4.0, 4.0, 1362.0, 13.5, 8.0, 7.0, 42.5, 1.0, 2.0, 0.0, 3.0, 1.0])
    #median0 = np.array([0.0, 2.0, 7.0, 5.0, 3539.0, 46.5, 2.0, 8.0, 38.5, 0.0, 1.0, 0.0, 5.0, 0.0])
        
    features, label = line[0], float(line[1])
    formated_features = []
    for i, value in enumerate(features):
        if value == '' and label == 1.0:
            formated_features.append(float(median1[i]))
        elif value == '' and label == 0.0:
            formated_features.append(float(median0[i]))
        else:
            if i < 13:
                formated_features.append(float(value)) 
            else:
                formated_features.append(value)
    return (formated_features, label)

In [662]:
toyRDDCached_train = toyRDD_train.map(parse).map(update_nans)
toyRDDCached_test = toyRDD_test.map(parse).map(update_nans)

In [663]:
meanClickthroughs = toyRDDCached_train.values().mean()

In [664]:
meanClickthroughs

0.25475295602315423

In [667]:
toyRDDCached_test.take(1)

[([1.0, 3.0, 5.0, 2.0, 2.0, 0.0, 55.0, 15.0, 23.0, 1.0, 11.0, 0.0, 0.0], 1.0)]

In [ ]:
# this is what I was working on to flatten outliers..


featureMeans = np.array(means)
featureStdev = np.array(stdevs)
upper_bounds = np.array(featureMeans + (featureStdev * 3))
lower_bounds = np.array(featureMeans - (featureStdev * 3))
# print("feature means", featureMeans)
# print("feature std devs", featureStdev)
# print("upper_bounds", upper_bounds)
# print("lower_bounds", lower_bounds)

def flatten_outliers(line):
    features = line[0]
    labels = line[1]
    for i in range(features):
        for j in range(upper_bounds):
            if i > j:
                features[i] = j
    for i in range(features):
        for k in range(lower_bounds):
            if i < k:
                features[i] = k
    return features, labels
            
# testRDD = toyRDDCached_train.mapValues(lambda x: x == ).take(1)
#testRDD.collect()


In [668]:
def standardize(dataRDD):
    """Standardize the data"""

    sc.broadcast(means)
    sc.broadcast(stdevs)
    
    featureMeans = np.array(means)
    featureStdev = np.array(stdevs)
       
    normedRDD = dataRDD.map(lambda x: ((x[0]-featureMeans)/featureStdev, x[1]))
    
    return normedRDD

In [669]:
# Normalize both the test and training sets
normedRDD_train = standardize(toyRDDCached_train).cache()
normedRDD_test = standardize(toyRDDCached_test).cache()

In [670]:
normedRDD_train.take(1)

[(array([ 0.14223638, -0.27238155, -0.18650095, -0.35274509, -0.28587587,
         -0.37326163,  0.97789862,  0.98357758, -0.14717497, -0.87465001,
          1.25780425, -0.26533126, -0.48695903]), 1.0)]

## 3. EDA & Discussion of Challenges

In [ ]:
# Get summary statitics
sample_df.iloc[:,0:21].describe(include = "all")

In [ ]:
# Get summary statistics
sample_df.iloc[:,21:39].describe(include = "all")

In [ ]:
# Take a look at histograms for each feature 
sample_numeric.hist(figsize=(23,15), bins=15)
#sample_numeric[FIELDS[:-1]].hist(figsize=(15,15), bins=15)
plt.show()

In [ ]:
# part b -  plot boxplots of each feature vs. the outcome

fig, ax_grid = plt.subplots(5, 3, figsize=(23,15))
y = sample_df['Label']
for idx, feature in enumerate(FIELDS[0:13]):
    x = sample_numeric[feature]
    sns.boxplot(x, y, ax=ax_grid[idx//3][idx%3], orient='h', linewidth=.5)
    ax_grid[idx//3][idx%3].invert_yaxis()
fig.suptitle("BoxPlots by Label", fontsize=15, y=0.9)
plt.show()

In [ ]:
# Look at correrlations across features

corr = sample_numeric[FIELDS[:13]].corr()
fig, ax = plt.subplots(figsize=(15, 13))
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(10, 240, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, center=0, linewidths=.5)
plt.title("Correlations between features")
plt.show()

## 4. Algorithm Implementation

### Following is a homegrown implementation of Logistic Regression

In [671]:
# Define the baseline model
BASELINE_1 = np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
BASELINE_2 = np.array([meanClickthroughs, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [672]:
# Compute log loss for Logisitc Regression
def LRLoss(cachedRDD, W):
    
    augmentedData = cachedRDD.map(lambda x: (np.append([1.0], x[0]), x[1]))

    loss = augmentedData.map(lambda x: -x[1] * np.log(1/ (1 + np.exp(-(np.dot(x[0],W))))) - (1-x[1]) * np.log(1-(1/ (1 + np.exp(-(np.dot(x[0],W))))))).mean()
    
    return loss

In [673]:
#Using meanClickthroughs as our bias term improves loss significantly (it will be used in our baseline)

train_loss_baseline1 = LRLoss(normedRDD_train, BASELINE_1)
print("Train Loss using 0 for baseline", train_loss_baseline1)

train_loss_baseline2 = LRLoss(normedRDD_train, BASELINE_2)
print("Train Loss using meanClickthroughs as baseline", train_loss_baseline2)


Train Loss using 0 for baseline 1.0585087314950672
Train Loss using meanClickthroughs as baseline 0.7637151310692348


In [674]:
BASELINE = np.array([meanClickthroughs, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [675]:
# Compute the gradient for logistic regression (one step)

def GDUpdate(dataRDD, W, learningRate):
    
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1])).cache()
        
    grad = augmentedData.map(lambda x: np.dot(x[0], ((1/ (1 + np.exp(-(np.dot(x[0],W))))) - x[1]))).mean()

    new_model = W - (learningRate * grad)

    return new_model

In [676]:
# Review Gradient Descent steps
nSteps = 3
model = BASELINE
#learningRate = .1
print(f"BASELINE:  Loss = {LRLoss(normedRDD_train,model)}")
for idx in range(nSteps):
    print("----------")
    print(f"STEP: {idx+1}")
    model = GDUpdate(normedRDD_train, model, .1)
    loss = LRLoss(normedRDD_train, model)
    print(f"Loss: {loss}")
    print(f"Model: {[round(w,3) for w in model]}")

BASELINE:  Loss = 0.7637151310692348
----------
STEP: 1
Loss: 0.746340881015326
Model: [0.224, 0.006, 0.001, 0.001, -0.001, -0.005, -0.001, 0.003, -0.003, 0.001, 0.025, 0.006, 0.008, -0.004]
----------
STEP: 2
Loss: 0.7302502404560913
Model: [0.194, 0.012, 0.003, 0.002, -0.001, -0.01, -0.001, 0.007, -0.006, 0.002, 0.049, 0.012, 0.015, -0.006]
----------
STEP: 3
Loss: 0.7152903868179628
Model: [0.165, 0.018, 0.004, 0.003, -0.002, -0.015, -0.002, 0.009, -0.008, 0.003, 0.072, 0.017, 0.022, -0.009]


In [677]:
# Perform one Gradient Descent update with lasso regularization

def GDUpdate_Lasso(dataRDD, W, learningRate, regParam):

    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1])).cache()
    #new_model = None    
    wReg = np.sign(W)
    wReg[0] = 0  #set bias to zero
       
    grad = augmentedData.map(lambda x: np.dot(x[0], ((1/ (1 + np.exp(-(np.dot(x[0],W))))) - x[1]))).mean() + (wReg * regParam)
    new_model = W - (learningRate * grad)
    
    return new_model

In [678]:
# Take a look at a few Gradient Descent steps
nSteps = 5
model = BASELINE
#learningRate = .1
#regParam = .1
print(f"BASELINE:  Loss = {LRLoss(normedRDD_train,model)}")
for idx in range(nSteps):
    print("----------")
    print(f"STEP: {idx+1}")
    model = GDUpdate_Lasso(normedRDD_train, model, .1, .1)
    loss = LRLoss(normedRDD_train, model)
    print(f"Loss: {loss}")
    print(f"Model: {[round(w,3) for w in model]}")

BASELINE:  Loss = 0.7637151310692348
----------
STEP: 1
Loss: 0.746340881015326
Model: [0.224, 0.006, 0.001, 0.001, -0.001, -0.005, -0.001, 0.003, -0.003, 0.001, 0.025, 0.006, 0.008, -0.004]
----------
STEP: 2
Loss: 0.7365672898524418
Model: [0.194, 0.002, -0.007, -0.008, 0.009, -0.0, 0.009, -0.003, 0.004, -0.008, 0.039, 0.002, 0.005, 0.004]
----------
STEP: 3
Loss: 0.7239573716740647
Model: [0.165, -0.002, 0.004, 0.007, -0.003, 0.005, -0.002, 0.01, -0.01, 0.004, 0.052, -0.003, 0.002, -0.011]
----------
STEP: 4
Loss: 0.7119064419227186
Model: [0.137, 0.014, -0.004, -0.004, 0.007, -0.01, 0.007, 0.003, -0.002, -0.005, 0.066, 0.013, -0.001, -0.003]
----------
STEP: 5
Loss: 0.7015280179148989
Model: [0.109, 0.009, 0.007, 0.009, -0.004, -0.005, -0.004, -0.004, 0.005, 0.006, 0.078, 0.008, 0.016, 0.004]


In [709]:
def get_scores(dataRDD, W, threshold):
    """Assess peformance of current models"""
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1]))
        
    sc.broadcast(W)
    sc.broadcast(threshold)

    def predict_label(line):
        features = line[0]
        label = line[1]
        pred = float(0.0)
        z = np.dot(features,W)
        prob = float(1 / (1 + np.exp(-z)))
        if prob > threshold:
            pred = float(1.0)
        else:
            pred = float(0.0)
        return label, pred
        
    def map_accuracy(line):
        label_actual = line[0]        
        label_pred = line[1]
        
        if label_actual == 1.0 and label_pred == 1.0:
            return "TP", 1.0
        elif label_actual == 1.0 and label_pred == 0.0:
            return "FN", 1.0
        elif label_actual == 0.0 and label_pred == 0.0:
            return "TN", 1.0
        else:
            if label_actual == 0.0 and label_pred == 1.0:
                return "FP", 1.0
    
    scores = augmentedData.map(predict_label).map(map_accuracy).reduceByKey(lambda x, y: x + y)
    
    return scores

In [710]:
#new_model = [0.773, 0.006, 0.008, 0.008, 0.005, -0.009, 0.006, 0.005, -0.007, -0.007, 0.105, -0.001, 0.011, -0.011]
# train_scores_test = get_scores(normedRDD_train, new_model, .5).collect()
# print(train_scores)
test_model = GDUpdate_Lasso(normedRDD_train, BASELINE, .1, .1)


In [711]:
Get_Scores = get_scores(normedRDD_train, BASELINE, .5).collect()
Get_Scores

[('FP', 27165.0), ('TP', 9286.0)]

In [708]:
Accuracy =  assess_performance(Get_Scores)
Accuracy


0.7666181997750404

In [712]:
# Assess performance of current models. Takes "train_scores" as input.

def assess_performance(scores):

    TP, FN, TN, FP = 0.0, 0.0, 0.0, 0.0

    for i in train_scores:
        if i[0] == 'TP':
            TP = i[1]
        elif i[0] == 'FN':
            FN = i[1]
        elif i[0] == 'TN':
            TN = i[1]
        else:
            if i[0] == 'FP':
                FP = i[1]

    if TP != 0.0:
        precision = TP/(TP + FP)
        recall = TP/(TP + FN)
        f1_score = 2*((precision*recall)/(precision+recall))
        accuracy = (TP+TN)/(TP+TN+FP+FN)   
    else:
        precision = 0.0
        recall = 0.0
        f1_score = 0.0
        accuracy = 0.0

#     print("True Positives=", TP)
#     print("False Negatives=", FN)
#     print("True Negatives=", TN)
#     print("False Positives=", FP)
#     print("Precision=", precision)
#     print("Recall=", recall)
#     print("F1_score=", f1_score)
#     print('Accuracy=', accuracy)
    
    return accuracy
        

In [686]:
# Run Logistic Regression gradient descent function and capture accuracy for each model
def GradientDescent(trainRDD, testRDD, wInit, nSteps = 10, 
                    learningRate = 0.1, verbose = True):
    """
    Perform nSteps iterations of Logistic Regression gradient descent and 
    track loss on a test and train set. Return lists of
    test/train loss and the models themselves.
    """
    # initialize lists to track model performance
    train_history, test_history, model_history = [], [], []
    
    # perform n updates & compute test and train loss after each
    model = wInit
    for idx in range(nSteps): 
        
        model = GDUpdate_Lasso(trainRDD, model, .1, .1)
        training_loss = LRLoss(trainRDD, model)
        test_loss = LRLoss(testRDD, model)
        
        #training_scores = get_scores(trainRDD, model, .5)
        #testing_scores = get_scores(testRDD, model, .5)
        
        #train_accuracy = assess_performance(training_scores)
        #test_accuracy = assess_performance(testing_scores)
        
        # keep track of accuracy for plotting
        train_history.append(training_loss)
        test_history.append(test_loss)
        model_history.append(model)
        
        # console output 
        if verbose:
            print("----------")
            print(f"STEP: {idx+1}")
            print(f"training accuracy: {training_loss}")
            print(f"test accuracy: {test_loss}")
            print(f"Model: {[round(w,3) for w in model]}")
    return train_history, test_history, model_history

In [687]:
# run 50 iterations (RUN THIS CELL AS IS)
wInit = BASELINE
#trainRDD, testRDD = normedRDD.randomSplit([0.8,0.2], seed = 2018)
start = time.time()
Train_Loss, Test_Loss, models = GradientDescent(normedRDD_train, normedRDD_test, wInit, nSteps = 3)
print(f"\n... trained {len(models)} iterations in {time.time() - start} seconds")

----------
STEP: 1
training accuracy: 0.746340881015326
test accuracy: 0.7438235953159592
Model: [0.224, 0.006, 0.001, 0.001, -0.001, -0.005, -0.001, 0.003, -0.003, 0.001, 0.025, 0.006, 0.008, -0.004]
----------
STEP: 2
training accuracy: 0.7365672898524418
test accuracy: 0.7345152531544128
Model: [0.194, 0.002, -0.007, -0.008, 0.009, -0.0, 0.009, -0.003, 0.004, -0.008, 0.039, 0.002, 0.005, 0.004]
----------
STEP: 3
training accuracy: 0.7239573716740647
test accuracy: 0.7221483167249952
Model: [0.165, -0.002, 0.004, 0.007, -0.003, 0.005, -0.002, 0.01, -0.01, 0.004, 0.052, -0.003, 0.002, -0.011]

... trained 3 iterations in 58.11057186126709 seconds


In [ ]:
# plot error curves - RUN THIS CELL AS IS
def plotErrorCurves(trainLoss, testLoss, title = None):
    """
    Helper function for plotting.
    Args: trainLoss (list of MSE) , testLoss (list of MSE)
    """
    fig, ax = plt.subplots(1,1,figsize = (16,8))
    x = list(range(len(train_Accuracy)))[1:]
    ax.plot(x, train_Accuracy[1:], 'k--', label='Training Accuracy')
    ax.plot(x, test_Accuracy[1:], 'r--', label='Test Accuracy')
    ax.legend(loc='upper right', fontsize='x-large')
    plt.xlabel('Number of Iterations')
    plt.ylabel('Accuracy')
    if title:
        plt.title(title)
    plt.show()

## 5. Application of Course Concepts